### Schiebeparkplatz

[Lösungsidee](schiebeparkplatz.pdf)

<img src='schiebeparkplatz.png' width='300'>

Die Beispieldaten von 0-5

In [20]:
nr = 0
f = open('./beispieldaten/parkplatz'+str(nr)+'.txt')
print(f.read())

A G
2
H 2
I 5



Lesen der ersten Eingabezeile. Damit wird die obere Wagenreihe modelliert

In [21]:
f = open('./beispieldaten/parkplatz'+str(nr)+'.txt')
c1, c2 = f.readline().split()
oben = [chr(c) for c in range(ord(c1), ord(c2)+1)]    # obere Wagenreihe als Liste mit Buchstaben
oben
 

['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [22]:
n = int(f.readline())   # anzahl querstehender Wagen                 
quer = []               # Liste mit Namen und Position der querstehenden Wagen
for i in range(n):
    name, pos = f.readline().split()
    quer.append((name,int(pos)))
    
quer

[('H', 2), ('I', 5)]

In [23]:
# Konstruktion des startstates als Tuple
startstate = ['.']*len(oben)
for c,k in quer:
    startstate[k]=c
    startstate[k+1]=c
    
startstate = tuple(startstate)
startstate

('.', '.', 'H', 'H', '.', 'I', 'I')

In [24]:
# Zur Visualisierung der Situation
def show(state):
    print(*oben)
    print(*state)
    
show(startstate)

A B C D E F G
. . H H . I I


Zur Suche in dem Baum der Möglichkeiten benötigen wir die Funktionen *goaltest* und *nextstates*.

In [25]:
wagenIdx = 5    # globale Variable, Index des Wagens, der rausgefahren werden soll

def goaltest(state):
    return state[wagenIdx] == '.'

In [26]:
querwagen = [c[0] for c in quer]      # Liste mit den Buchstaben der querstehenden Wagen

def nextstates(state):
    tmp = []
    for c in querwagen:
        k = state.index(c)
        new_state = list(state)
        while k+2 < len(state) and state[k+2]=='.':   # solange rechts Platz ist
            new_state[k]='.'                          # verschiebe die beiden Zeichen in new_state                     
            new_state[k+2]=c                          # um eins nach rechts
            tmp.append(tuple(new_state))              # speichere den Zustand als möglichen Folgezustand
            k+=1
            
        k = state.index(c)                            
        new_state = list(state)
        while k-1 >= 0 and new_state[k-1]=='.':       # solange links Platz ist
            new_state[k-1]=c
            new_state[k+1]='.'
            tmp.append(tuple(new_state))
            k-=1
    return tmp

In [27]:
nextstates(startstate)

[('.', '.', '.', 'H', 'H', 'I', 'I'),
 ('.', 'H', 'H', '.', '.', 'I', 'I'),
 ('H', 'H', '.', '.', '.', 'I', 'I'),
 ('.', '.', 'H', 'H', 'I', 'I', '.')]

In [28]:
from collections import deque
def bfs(startstate):
    frontier =  deque([startstate])
    prev = {startstate:None}
    if goaltest(startstate):
            return prev, startstate
    while frontier:
        state = frontier.popleft()
        for v in nextstates(state):
            if v not in prev:
                prev[v] = state
                if goaltest(v):
                    return prev, v
                frontier.append(v)
 

In [29]:
prev, goalstate = bfs(startstate)
goalstate

('.', 'H', 'H', 'I', 'I', '.', '.')

In [30]:
prev

{('.', '.', 'H', 'H', '.', 'I', 'I'): None,
 ('.', '.', '.', 'H', 'H', 'I', 'I'): ('.', '.', 'H', 'H', '.', 'I', 'I'),
 ('.', 'H', 'H', '.', '.', 'I', 'I'): ('.', '.', 'H', 'H', '.', 'I', 'I'),
 ('H', 'H', '.', '.', '.', 'I', 'I'): ('.', '.', 'H', 'H', '.', 'I', 'I'),
 ('.', '.', 'H', 'H', 'I', 'I', '.'): ('.', '.', 'H', 'H', '.', 'I', 'I'),
 ('.', 'H', 'H', '.', 'I', 'I', '.'): ('.', 'H', 'H', '.', '.', 'I', 'I'),
 ('.', 'H', 'H', 'I', 'I', '.', '.'): ('.', 'H', 'H', '.', '.', 'I', 'I')}

In [31]:
def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]        
    path.reverse()
    return path

In [32]:
path = reconstructPath(prev,goalstate)
path

[('.', '.', 'H', 'H', '.', 'I', 'I'),
 ('.', 'H', 'H', '.', '.', 'I', 'I'),
 ('.', 'H', 'H', 'I', 'I', '.', '.')]

Um die Abfolge des Pfades in Worten wiederzugeben, definieren wir uns eine Funktion *schieb*

In [33]:
def schieb(s1, s2):
    for c in querwagen:
        i1 = s1.index(c)
        i2 = s2.index(c)
        if i1 > i2:
            return f'{c}: {i1-i2} nach links ' 
        if i1 < i2:
            return f'{c}: {i2-i1} nach rechts' 

In [34]:
for i in range(len(path)-1):
    print(schieb(path[i],path[i+1]))

H: 1 nach links 
I: 2 nach links 


In einer Schleife lassen ermitteln wir die Ergebnisse für alle Wagen

In [35]:
for w in oben:
    wagenIdx = ord(w)-ord('A')
    prev, state = bfs(startstate)
    path = reconstructPath(prev, state)
    print(w,': ',end='')
    for i in range(len(path)-1):
        print(schieb(path[i],path[i+1]),end='  ')
    print()

A : 
B : 
C : H: 1 nach rechts  
D : H: 1 nach links   
E : 
F : H: 1 nach links   I: 2 nach links   
G : I: 1 nach links   


### Das vollständige Programm

In [37]:
from collections import deque

def goaltest(state):
    return state[wagenIdx] == '.'

def nextstates(state):
    tmp = []
    for c in querwagen:
        k = state.index(c)
        new_state = list(state)
        while k+2 < len(state) and state[k+2]=='.':   # solange rechts Platz ist
            new_state[k]='.'                          # verschiebe die beiden Zeichen in new_state                     
            new_state[k+2]=c                          # um eins nach rechts
            tmp.append(tuple(new_state))              # speichere den Zustand als möglichen Folgezustand
            k+=1
            
        k = state.index(c)                            
        new_state = list(state)
        while k-1 >= 0 and new_state[k-1]=='.':       # solange links Platz ist
            new_state[k-1]=c
            new_state[k+1]='.'
            tmp.append(tuple(new_state))
            k-=1
    return tmp

def bfs(startstate):
    frontier = deque([startstate])
    prev = {startstate:None}
    if goaltest(startstate):
            return prev, startstate
    while frontier:
        state = frontier.popleft()
        for v in nextstates(state):
            if v not in prev:
                prev[v] = state
                if goaltest(v):
                    return prev, v
                frontier.append(v)
                
def reconstructPath(prev,goalstate):
    state = goalstate
    path = []
    while state is not None:
        path.append(state)
        state = prev[state]        
    path.reverse()
    return path

def schieb(s1, s2):
    for c in querwagen:
        i1 = s1.index(c)
        i2 = s2.index(c)
        if i1 > i2:
            return f'{c}: {i1-i2} nach links ' 
        if i1 < i2:
            return f'{c}: {i2-i1} nach rechts' 

# Obere Wagenreihe        
nr = 2
f = open('./beispieldaten/parkplatz'+str(nr)+'.txt')
c1, c2 = f.readline().split()
oben = [chr(c) for c in range(ord(c1), ord(c2)+1)]  

# Untere Wagenreihe
n = int(f.readline())   # anzahl querstehender Wagen                 
quer = []               # Liste mit Namen und Position der querstehenden Wagen
for i in range(n):
    name, pos = f.readline().split()
    quer.append((name,int(pos)))

querwagen = [c[0] for c in quer]   # Liste mit den Buchstaben der querstehenden Wagen

# Konstruktion startstate
startstate = ['.']*len(oben)
for c,k in quer:
    startstate[k]=c
    startstate[k+1]=c
startstate = tuple(startstate)

# Breitensuche und Ausgabe Ergebnisse
for w in oben:
    wagenIdx = ord(w)-ord('A')
    prev, state = bfs(startstate)
    path = reconstructPath(prev, state)
    print(w,': ',end='')
    for i in range(len(path)-1):
        print(schieb(path[i],path[i+1]),end='  ')
    print()

 

A : 
B : 
C : O: 1 nach rechts  
D : O: 1 nach links   
E : 
F : O: 1 nach links   P: 2 nach links   
G : P: 1 nach links   
H : R: 1 nach rechts  Q: 1 nach rechts  
I : P: 1 nach links   Q: 1 nach links   
J : R: 1 nach rechts  
K : P: 1 nach links   Q: 1 nach links   R: 1 nach links   
L : 
M : P: 1 nach links   Q: 1 nach links   R: 1 nach links   S: 2 nach links   
N : S: 1 nach links   
